In [26]:
from flask import Flask, render_template, request
import pickle
import numpy as np
import pandas as pd
import os

In [27]:
app=Flask(__name__)
model=pickle.load(open(r'model.pkl','rb'))


In [28]:
@app.route('/')
def home():
    return render_template('home.html')
@app.route('/predict')
def predict():
    return render_template('predict.html')
@app.route('/output',methods=['POST'])
def output():
    input_feature=[x for x in request.form.values() ]
    input_feature=[np.array(input_feature)]
    print(input_feature)
    names=['Gender','Married','Dependents','Education','Self_Employed','ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History','Property_Area']
    data=pd.DataFrame(input_feature,columns=names)
    print(data)
    
    prediction=model.predict(data)
    prediction=int(prediction)
    print(type(prediction))
    
    if(prediction==0):
        return render_template('submit.html',result="Loan will Not be Approved")
        
    else:
        return render_template('submit.html',result="Loan will be Approved")
        

In [29]:
if __name__=='__main__':
    port=int(os.environ.get('PORT',5000))
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Nov/2022 16:09:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2022 16:09:21] "GET /predict HTTP/1.1" 200 -
C:\Users\kirut\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
127.0.0.1 - - [19/Nov/2022 16:09:26] "POST /output HTTP/1.1" 200 -


[array(['1', '1', '0', '0', '0', '4000', '1000', '120', '360', '0', '0'],
      dtype='<U4')]
  Gender Married Dependents Education Self_Employed ApplicantIncome  \
0      1       1          0         0             0            4000   

  CoapplicantIncome LoanAmount Loan_Amount_Term Credit_History Property_Area  
0              1000        120              360              0             0  
<class 'int'>


In [30]:
from flask import Flask, render_template, request, jsonify

import requests
import json
import numpy as np
import pandas as pd



In [31]:
API_KEY = "iFY0zXPpW9WAKkEngxaszWLAMJARuVJzMZgH53lU3o7o"
token_response = requests.post('https://iam.cloud.ibm.com/identity/token', data={"apikey":API_KEY, "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'})
mltoken = token_response.json()["access_token"]

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}



In [32]:
app=Flask(__name__)

@app.route('/home')
def home():
    return render_template('home.html')
@app.route('/predict')
def predict():
    return render_template('predict.html')
@app.route('/output',methods=['POST'])
def output():
    input_feature=[x for x in request.form.values()]
    input_feature=[np.array(input_feature)]
    print(input_feature)
    names=['Gender','Married','Dependents','Education','Self_Employed','ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History','Property_Area']
    data=pd.DataFrame(input_feature,columns=names)
    print(data)
    
    response_scoring = requests.post('https://us-south.ml.cloud.ibm.com/ml/v4/deployments/e858e595-92ed-44da-ba4a-f33d452c403c/predictions?version=2022-11-19', json=data,headers={'Authorization': 'Bearer ' + mltoken})
    
    prediction=int(response_scoring.json())
    if(prediction==0):
        return render_template('submit.html',result="Loan will Not be Approved")
        
    else:
        return render_template('submit.html',result="Loan will be Approved")

In [ ]:
if __name__=='__main__':
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Nov/2022 16:23:03] "GET / HTTP/1.1" 404 -
